In [1]:
# 먼저 GPU 서버를 켜놓는다. GPU 서버의 Embedding 모델을 활용해야 하기 때문.
# GPU_URL = "http://222.107.238.44:8109/v1" #GPU 서버 URL 및 포트, 형식: http://some_url:8109
GPU_URL = "http://localhost:8109/v1" #GPU 서버 URL 및 포트, 형식: http://some_url:8109
DB_URL = "here_law_admin:1234@3.36.85.129:5434/here_law" #DB URL. 형식: 계정이름:비밀번호@도메인:포트/사용할 DB 이름
API_KEY="sk-proj-1pGHnq8uFfaPqRQUWCdgT3BlbkFJ9uF478M8oyptDikjRI7o"

In [2]:
from agents.rag_agent import CaseRagAgent

In [3]:
rag_agent = CaseRagAgent(db_url=DB_URL, embedder_url=GPU_URL, api_key=API_KEY, temperature=0)

In [4]:
inputs = {"messages": [("user", "수면 관련해서 문제가 되었던 판례를 찾아봐바.")]}
# rag_agent._print_stream(inputs)
result = rag_agent(inputs)

In [5]:
resultarray = []
for s in result:
    resultarray.append(s)

In [ ]:
resultarray